In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.io_model
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path
import tensorflow as tf

In [ ]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/japan_cup/'


In [ ]:
df = rd.read_horse_csv(DATA_PATH)

In [ ]:
df

In [ ]:
# 学習に用いるデータセットの作成
x = np.array(df[["frame_number", "burden_weight"]]).astype(np.float32)
del df

In [ ]:
model = io_model.read_model('')

In [ ]:
model.predict_proba(x)